### chatcompletition, Prompt Template, LLM api call with Open Source LLMs,with local storage

In [1]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)  

Index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [4]:
def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

### Mistral-7B openSource LLM

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [6]:
from huggingface_hub import login

In [7]:
login(os.getenv("HF_zoomcamp"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /teamspace/studios/this_studio/.cache/huggingface/token
Login successful


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.3", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3", padding_side="left")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
generator = pipeline("text-generation", model = model, tokenizer = tokenizer)

In [16]:
def build_prompt(query, search_results):
    prompt_template = """ You are an expert in machine learning and MLOps, assisting a junior engineer. Your task is to answer the following question using only the provided context from the FAQ database. Do not include any information that is not present in the context. If the context does not provide an answer to the question, respond with "Not FOUND in the context given" and provide a brief explanation.

QUESTION:{question}
CONTEXT:{context}
Answer:
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"{doc['question']}\n{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm_call(prompt):
    response = generator(prompt, max_length=2048, do_sample=True, temperature=0.7,top_p = 0.95, num_return_sequences=1)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()

In [19]:
rag("How can I use Kafka?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'The reason you are getting this error is because you are trying to connect to an SSL server with Python 2.x. Python 2.x does not support SSL connections.\n\nYou should upgrade to Python 3.x to resolve the problem.'

In [20]:
print(_)

The reason you are getting this error is because you are trying to connect to an SSL server with Python 2.x. Python 2.x does not support SSL connections.

You should upgrade to Python 3.x to resolve the problem.
